In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define image size & batch size
IMG_SIZE = (224, 224)  # Resize to 224x224
BATCH_SIZE = 32

# Define paths to the dataset
train_dir = "sample_data/train"
test_dir = "sample_data/test"

# Apply Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values (0-1)
    rotation_range=20,      # Rotate images randomly
    width_shift_range=0.2,  # Shift width
    height_shift_range=0.2, # Shift height
    shear_range=0.2,        # Shear transformation
    zoom_range=0.2,         # Zoom-in/out
    horizontal_flip=True,   # Flip images horizontally
    validation_split=0.2    # 20% for validation
)

# Load Training Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Use 'binary' for two classes
    subset='training'  # Training set
)

# Load Validation Data
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # Validation set
)

# Load Test Data (without augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Data preprocessing complete!")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Train the model
cnn_model.fit(train_generator, 
              validation_data=val_generator, 
              epochs=10)

# Save model
cnn_model.save("cnn_brain_tumor_model.h5")

print("CNN model training complete!")


In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained Xception model (without top layers)
base_model = Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model (don't train initial layers)
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')  # 4 classes

# Create new model
xception_model = Model(inputs=base_model.input, outputs=x)

# Compile the model
xception_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Train the model
xception_model.fit(train_generator, 
                   validation_data=val_generator, 
                   epochs=10)

# Save model
xception_model.save("xception_brain_tumor_model.h5")

print("Xception model training complete!")


In [3]:
# Evaluate CNN Model
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(test_generator)
print(f"CNN Model - Test Accuracy: {cnn_test_acc:.2f}, Test Loss: {cnn_test_loss:.2f}")

# Evaluate Xception Model
xception_test_loss, xception_test_acc = xception_model.evaluate(test_generator)
print(f"Xception Model - Test Accuracy: {xception_test_acc:.2f}, Test Loss: {xception_test_loss:.2f}")


C:\Users\DEVICES\Desktop\Brain_Tumor_Classification\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


OverflowError: Python int too large to convert to C long